In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.3 MB/s eta 0:00:00


In [3]:
import numpy as py
import pandas as pd
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, Dropout, LSTM, GRU, Input, GlobalMaxPooling1D, LayerNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras. preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pyvi import ViTokenizer
from pyvi import ViUtils

In [4]:
data = pd. read_excel("/content/drive/MyDrive/Colab Notebooks/DATA/Datacomment.xlsx")
data.head(10)
data.shape

(1649, 3)

In [5]:
sentiment_data = pd.DataFrame({'input':data['Comment'], 'label':data['Label']})
sentiment_data = sentiment_data.dropna()
sentiment_data = sentiment_data.reset_index(drop=True)
sentiment_data.head()

,input,label
0,"Không nên mua chuột cua Logitech , vì dùng nó ...",Pos
1,Nói thiệt là mình thì thì chuột nào mình cũng ...,Neg
2,Xai chuot so nhat bi double click .,Neu
3,Cơ bản là thiết kế ôm chuột chưa đã Như hiện g...,Pos
4,Đang dùng mx 1 . Cũng ngon nhưng chưa đầy năm ...,Neg


In [6]:
#xóa dữ liệu trùng lặp
sentiment_data.drop_duplicates()
sentiment_data.shape

(1649, 2)

In [7]:
input_data = sentiment_data['input'].values
input_label = sentiment_data['label'].values

label_dict = {'Neg':0,'Pos':1, 'Neu': 2}

input_pre = []
label_with_accent = []
for idx, dt in enumerate(input_data):
  #input_text_pre = list(tf.keras.preprocessing.text.text_to_word_sequence(dt))
  input_text_pre = list(tf.keras.preprocessing.text.text_to_word_sequence(str(dt)))
  input_text_pre = " ".join(input_text_pre)
  input_text_pre_no_accent = str(ViUtils.remove_accents(input_text_pre).decode("utf-8"))
  input_text_pre_accent = ViTokenizer.tokenize(input_text_pre)
  input_text_pre_no_accent = ViTokenizer.tokenize(input_text_pre_no_accent)
  input_pre.append(input_text_pre_accent)
  input_pre.append(input_text_pre_no_accent)
  label_with_accent.append(input_label[idx])
  label_with_accent.append(input_label[idx])

In [8]:
negative_count = len(data[data['Label'] == 'Neg'])
print("Số lượng nhãn Negative là:", negative_count)

positive_count = len(data[data['Label'] == 'Pos'])
print("Số lượng nhãn Positive là:", positive_count)

neutral_count = len(data[data['Label'] == 'Neu'])
print("Số lượng nhãn Neutral là:", neutral_count)

Số lượng nhãn Negative là: 593
Số lượng nhãn Positive là: 552
Số lượng nhãn Neutral là: 504


In [9]:
# Chuyển đổi tokenizer bộ dữ liệu và lưu vào file
label_idx = [label_dict[i] for i in label_with_accent]
label_tf = tf.keras.utils.to_categorical(label_idx, num_classes=3, dtype='float32')

# khởi tạo một đối tượng tokenizer_data từ lớp Tokenizer trong Keras để mã hóa
# và chuyển đổi dữ liệu văn bản thành chuỗi số nguyên.
tokenizer_data = Tokenizer(oov_token='<OOV>', filters = '', split = ' ')

# sử dụng phương thức fit_on_texts() của tokenizer_data để thực hiện mã hóa
# từ vựng và xây dựng từ điển từ dữ liệu input_pre. Điều này giúp xác định
# các từ duy nhất và gán một số nguyên duy nhất cho mỗi từ.
tokenizer_data.fit_on_texts(input_pre)

# sử dụng phương thức texts_to_sequences() của tokenizer_data để chuyển đổi
# dữ liệu văn bản input_pre thành các chuỗi số nguyên tương ứng với từng
# từ trong từ điển
tokenizer_data_text = tokenizer_data.texts_to_sequences(input_pre)

# sử dụng hàm pad_sequences() để đệm các chuỗi số nguyên trong
# tokenizer_data_text thành cùng một độ dài (maxlen = 120).
# Điều này đảm bảo rằng các chuỗi đầu vào có cùng kích thước
vec_data = pad_sequences(tokenizer_data_text, padding='post', maxlen = 120)

pickle.dump(tokenizer_data, open("tokenizer_data.pkl", "wb"))
# Lưu model tokenizer_data.pkl vào đường dẫn /content/drive/My Drive/CD
with open("/content/drive/MyDrive/tokenizer_data.pkl", "wb") as file:
    pickle.dump(tokenizer_data, file)

print("input data.shape", vec_data.shape)
data_vocab_size = len(tokenizer_data.word_index)+1
print("data_vocab_size: ", data_vocab_size)

# Chia tập dữ liệu thành 3 phần train, val, test
X_train, X_val, y_train, y_val = train_test_split(vec_data, label_tf, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(vec_data, label_tf, test_size=0.1, random_state = 42)
print("Training sample: ", len(X_train))
print("Validation sample: ", len(X_val))
print("Test sample: ", len(X_test))

input data.shape (3298, 120)
data_vocab_size:  5158
Training sample:  2968
Validation sample:  660
Test sample:  330


In [10]:
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Bidirectional, LSTM
from keras.models import Model
from keras.optimizers import Adam

In [11]:
def generate_model():
  # khai báo ngưỡng dropout, đại diện cho tỷ lệ các đơn vị bị bỏ qua
  # trong quá trình huấn luyện để tránh việc quá khớp (overfitting)
  dropout_threshold = 0.3

  # gán kích thước của từ điển từ dữ liệu cho biến input_dim
  input_dim = data_vocab_size

  # gán kích thước đầu ra của lớp nhúng (embedding) cho biến output_dim.
  # Trong trường hợp này, các từ sẽ được biểu diễn bằng một vector 32 chiều.
  output_dim = 32

  # gán độ dài đầu vào cho biến input_length. đảm bảo rằng độ dài của các chuỗi
  # đầu vào sẽ được đệm (padding) hoặc cắt (truncation) để có cùng kích thước
  input_length = 120

  # khởi tạo một đối tượng GlorotNormal từ module initializers trong
  # thư viện Keras để sử dụng làm trình khởi tạo trọng số cho mạng nơ-ron.
  initializer = tf.keras.initializers.GlorotNormal()

  # khai báo lớp đầu vào với kích thước đầu vào là input_length
  input_layer = Input(shape=input_length)

  # thực hiện lớp nhúng (embedding) ánh xạ các từ sang các vector dựa trên
  # input_dim và output_dim. embeddings_initializer được sử dụng để khởi tạo
  # ma trận trọng số của lớp nhúng
  feature = Embedding(input_dim=input_dim, output_dim=output_dim,
                      input_length=input_length,
                      embeddings_initializer="GlorotNormal")(input_layer)

  # Sử dụng mạng nơ ron tích chập trích xuất các đặc trưng từ đầu vào đã nhúng.
  # Các lớp này được sử dụng để tạo một đặc trưng từ ngữ cảnh và cấu trúc của câu
  cnn_feature = Conv1D(filters=32, kernel_size=3, padding='same',
                       activation='relu')(feature)
  cnn_feature = MaxPooling1D()(cnn_feature)
  cnn_feature = Dropout(dropout_threshold)(cnn_feature)
  cnn_feature = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(cnn_feature)
  cnn_feature = MaxPooling1D()(cnn_feature)
  cnn_feature = LayerNormalization()(cnn_feature)
  cnn_feature = Dropout(dropout_threshold)(cnn_feature)

  # chuỗi các lớp LSTM hai chiều (Bidirectional LSTM) để trích xuất
  # các đặc trưng từ các chuỗi đầu vào. Lớp này giúp mô hình có khả năng học
  # được các phụ thuộc dài hạn trong dữ liệu chuỗi.
  bi_lstm_feature = Bidirectional(LSTM(units=32, dropout=dropout_threshold,
                                       return_sequences=True,
                                       kernel_initializer=initializer),
                                  merge_mode = 'concat')(feature)
  bi_lstm_feature = MaxPooling1D()(bi_lstm_feature)

  bi_lstm_feature = Bidirectional(LSTM(units=32, dropout=dropout_threshold,
                                       return_sequences=True,
                                       kernel_initializer=initializer),
                                  merge_mode = 'concat')(bi_lstm_feature)
  bi_lstm_feature = MaxPooling1D()(bi_lstm_feature)
  bi_lstm_feature = LayerNormalization()(bi_lstm_feature)

  # kết hợp hai tập đặc trưng từ các lớp tích chập và LSTM thành
  # một tập đặc trưng duy nhất bằng cách kết hợp chúng theo chiều thứ hai.
  combine_feature = tf.keras.layers.Concatenate()([cnn_feature, bi_lstm_feature])
  combine_feature = GlobalMaxPooling1D()(combine_feature)
  combine_feature = LayerNormalization()(combine_feature)

  # chuỗi các lớp mạng nơ-ron tiếp theo để xử lý tập đặc trưng đã kết hợp.
  # Các lớp này thực hiện các phép biến đổi phi tuyến để tạo ra dự đoán cuối cùng.
  classifier = Dense(90, activation = 'relu')(combine_feature)
  classifier = Dropout(0.3)(classifier)
  classifier = Dense(70, activation = 'relu')(classifier)
  classifier = Dropout(0.3)(classifier)
  classifier = Dense(50, activation = 'relu')(classifier)
  classifier = Dropout(0.3)(classifier)
  classifier = Dense(30, activation = 'relu')(classifier)
  classifier = Dropout(0.3)(classifier)
  classifier = Dense(3, activation = 'softmax')(classifier)

  model = tf.keras.Model(inputs = input_layer, outputs= classifier)

  return model

model = generate_model()
adam = Adam(learning_rate=0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 120)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 120, 32)              165056    ['input_1[0][0]']             
                                                                                                  
 conv1d (Conv1D)             (None, 120, 32)              3104      ['embedding[0][0]']           
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 60, 32)               0         ['conv1d[0][0]']              
 D)                                                                                           

In [12]:
# Huấn luyện mô hình máy học sử dụng mạng CNN-BiLSTM trên dữ liệu huấn luyện
# và dữ liệu kiểm tra (validation)

callback_model = tf.keras.callbacks.ModelCheckpoint('model_cnn_bilstm.h5', monitor='val_loss')
history = model.fit(x = X_train, y = y_train, validation_data = (X_val, y_val), epochs = 50, batch_size = 128, callbacks=[callback_model])

Epoch 1/50
24/24 [==============================] - 24s 500ms/step - loss: 1.1220 - accuracy: 0.3420 - val_loss: 1.0959 - val_accuracy: 0.3894
Epoch 2/50


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - 10s 417ms/step - loss: 1.1005 - accuracy: 0.3393 - val_loss: 1.0954 - val_accuracy: 0.3470
Epoch 3/50
24/24 [==============================] - 8s 323ms/step - loss: 1.1026 - accuracy: 0.3386 - val_loss: 1.0984 - val_accuracy: 0.3152
Epoch 4/50
24/24 [==============================] - 10s 433ms/step - loss: 1.0978 - accuracy: 0.3464 - val_loss: 1.0958 - val_accuracy: 0.3636
Epoch 5/50
24/24 [==============================] - 11s 470ms/step - loss: 1.0975 - accuracy: 0.3578 - val_loss: 1.0891 - val_accuracy: 0.3955
Epoch 6/50
24/24 [==============================] - 8s 349ms/step - loss: 1.0955 - accuracy: 0.3666 - val_loss: 1.0874 - val_accuracy: 0.3788
Epoch 7/50
24/24 [==============================] - 10s 421ms/step - loss: 1.0880 - accuracy: 0.3662 - val_loss: 1.0845 - val_accuracy: 0.3803
Epoch 8/50
24/24 [==============================] - 8s 340ms/step - loss: 1.0674 - accuracy: 0.4023 - val_loss: 1.0412 - val_accuracy: 0.4258
Epoch 9/50
24

In [13]:
# Kiểm tra độ chính xác của mô hình trên tập test
model.load_weights("model_cnn_bilstm.h5")
model.evaluate(X_test,y_test)

11/11 [==============================] - 1s 62ms/step - loss: 4.3881 - accuracy: 0.7273


[4.38805627822876, 0.7272727489471436]

In [14]:
labels = ["Neg","Pos", "Neu"]

y_pred = model.predict(X_val)  # Dự đoán trên tập validation
# y_pred = model.predict(X_test)  # Dự đoán trên tập test

21/21 [==============================] - 3s 50ms/step


In [15]:
import numpy as np

y_pred = [labels[np.argmax(pred)] for pred in y_pred]
y_true = [labels[np.argmax(true)] for true in y_val]  # Nhãn thực tế trên tập validation
# y_true = [labels[np.argmax(true)] for true in y_test]  # Nhãn thực tế trên tập test

In [16]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred, zero_division=1)
print(report)

              precision    recall  f1-score   support

         Neg       0.83      0.89      0.86       227
         Neu       0.84      0.82      0.83       196
         Pos       0.92      0.86      0.89       237

    accuracy                           0.86       660
   macro avg       0.86      0.86      0.86       660
weighted avg       0.86      0.86      0.86       660



In [17]:
def preprocess_raw_input(raw_input, tokenizer):
  input_text_pre = list(tf.keras.preprocessing.text.text_to_word_sequence(raw_input))
  input_text_pre = " ".join(input_text_pre)
  input_text_pre_accent = ViTokenizer.tokenize(input_text_pre)
  print("Text preprocessed: ", input_text_pre_accent)
  tokenized_data_text = tokenizer.texts_to_sequences([input_text_pre_accent])
  vec_data = pad_sequences(tokenized_data_text, padding='post', maxlen=120)
  return vec_data

def inference_model(input_feature, model):
  output = model(input_feature).numpy()[0]
  result = output.argmax()
  conf = float(output.max())
  label_dict = {'Tiêu cực':0, 'Tích cực':1, 'Trung lập':2}
  label = list(label_dict.keys())
  return label[int(result)], conf

def prediction(raw_input, tokenizer, model):
  input_model = preprocess_raw_input(raw_input, tokenizer_data)
  result, conf = inference_model(input_model, model)

  return result, conf

my_model = generate_model()
my_model = load_model('model_cnn_bilstm.h5')

with open(r"tokenizer_data.pkl", "rb") as input_file:
  my_tokenizer = pickle.load(input_file)

print(prediction("Ứng dụng bình thường", my_tokenizer, my_model))

Text preprocessed:  ứng_dụng bình_thường
('Trung lập', 0.9898321628570557)


In [18]:
# vòng lặp vô hạn để kiểm thử mô hình phân loại dựa vào đầu vào từ
# người dùng qua dòng lệnh. Người dùng sẽ nhập các đoạn văn bản để mô hình
# dự đoán lớp (sentiment) tương ứng cho mỗi đoạn văn bản đó.
# Người dùng có thể nhập "end" để kết thúc vòng lặp và thoát khỏi chương trình
while(True):
  text = input()
  if text == "end":
    break
  else:
    print(prediction(text, my_tokenizer, my_model)[0] + "\n")

Ứng dụng dùng tuyệt vời 
Text preprocessed:  ứng_dụng dùng tuyệt_vời
Tích cực

không nên mua vì nó dùng không bao giờ hư
Text preprocessed:  không nên mua vì nó dùng không bao_giờ hư
Tiêu cực

hay quá 
Text preprocessed:  hay quá
Tiêu cực

Dùng ổn nha
Text preprocessed:  dùng ổn nha
Tích cực

end


In [19]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.2 MB/s eta 0:00:00


In [25]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from pyvi import ViTokenizer
import pickle

def preprocess_raw_input(raw_input, tokenizer):
    input_text_pre = list(tf.keras.preprocessing.text.text_to_word_sequence(raw_input))
    input_text_pre = " ".join(input_text_pre)
    input_text_pre_accent = ViTokenizer.tokenize(input_text_pre)
    tokenized_data_text = tokenizer.texts_to_sequences([input_text_pre_accent])
    vec_data = pad_sequences(tokenized_data_text, padding='post', maxlen=120)
    return vec_data

def inference_model(input_feature, model):
    output = model(input_feature).numpy()[0]
    result = output.argmax()
    conf = float(output.max())
    label_dict = {'Tiêu cực': 0, 'Tích cực': 1, 'Trung lập': 2}
    label = list(label_dict.keys())
    return label[int(result)], conf

def prediction(raw_input, tokenizer, model):
    input_model = preprocess_raw_input(raw_input, tokenizer)
    result, conf = inference_model(input_model, model)
    return result, conf

# Tải model và tokenizer
@st.cache_resource
def load_model_and_tokenizer():
    model = load_model('model_cnn_bilstm.h5')
    with open("tokenizer_data.pkl", "rb") as input_file:
        tokenizer = pickle.load(input_file)
    return model, tokenizer

# Tải model và tokenizer
my_model, my_tokenizer = load_model_and_tokenizer()

# Giao diện Streamlit
st.title('Dự đoán Cảm xúc Văn bản Tiếng Việt')
st.write('Nhập văn bản của bạn vào ô dưới đây và nhận kết quả dự đoán cảm xúc.')

user_input = st.text_area('Nhập văn bản tại đây:')

if st.button('Dự đoán'):
    if user_input:
        result, conf = prediction(user_input, my_tokenizer, my_model)
        st.write(f'Kết quả: {result}')
        st.write(f'Độ tin cậy: {conf:.2f}')
    else:
        st.write('Vui lòng nhập văn bản để dự đoán.')


Writing app.py


In [ ]:
%%writefile app.py
import tensorflow as tf
import streamlit as st
import pandas as pd
import pickle
from keras.models import Model
from pyvi import ViTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

def preprocess_raw_input(raw_input, tokenizer):
  input_text_pre = list(tf.keras.preprocessing.text.text_to_word_sequence(raw_input))
  input_text_pre = " ".join(input_text_pre)
  input_text_pre_accent = ViTokenizer.tokenize(input_text_pre)
  print("Text preprocessed: ", input_text_pre_accent)
  tokenized_data_text = tokenizer.texts_to_sequences([input_text_pre_accent])
  vec_data = pad_sequences(tokenized_data_text, padding='post', maxlen=120)
  return vec_data

# def inference_model(input_feature, model):
#   output = model(input_feature).numpy()[0]
#   result = output.argmax()
#   conf = float(output.max())
#   label_dict = {'Tiêu cực':0, 'Tích cực':1, 'Trung lập':2}
#   label = list(label_dict.keys())
#   return label[int(result)], conf

# def prediction(raw_input, tokenizer, model):
#   input_model = preprocess_raw_input(raw_input, tokenizer_data)
#   result, conf = inference_model(input_model, model)

#   return result, conf
st.write("""
# Mô hình dự đoán sự hài lòng của khách hàng

Mô hình dự đoán lương dựa sự hài lòng của khách hàng hàng không
""")


with open(r"tokenizer_data.pkl", "rb") as input_file:
  my_tokenizer = pickle.load(input_file)

user_input = st.text_input('Nhập văn bản:')
st.write(user_input)
user_input_final = preprocess_raw_input(user_input, my_tokenizer)



my_model = load_model('model_cnn_bilstm.h5')


if user_input_final:
    pred = my_model.predict(user_input_final)[0]

    # Display prediction
    if pred == 'Tích cực':
        st.write('## Sự hài lòng của khách hàng: Tích cực')
    elif pred == 'Tiêu cực':
        st.write('## Sự hài lòng của khách hàng: Tiêu cực')
    else:
        st.write('## Sự hài lòng của khách hàng: Trung lập')

Overwriting app.py


In [24]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 0.886s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


   ╭────────────────────────────────────────────────────────────────╮
   │                                                                │
   │      New major version of npm available! 6.14.8 → 10.8.1       │
   │   Changelog: https://github.com/npm/cli/releases/tag/v10.8.1   │
   │               Run npm install -g npm to update!                │
   │ 

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.125.12.231
npx: installed 22 in 2.123s
your url is: https://major-hotels-cheer.loca.lt
